In [1]:
import os
from crowemi_trades.storage.s3_storage import S3Storage
from datetime import datetime, tzinfo
import polars as pl

stor = S3Storage(
    access_key=os.getenv("AWS_ACCESS_KEY_ID", None),
    secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY", None),
    region="us-west-2"
)
t = stor.read_parquet(bucket="crowemi-trades", key="C:EURUSD/minute/5/2023/04/20230412.parquet.gzip",)
print(t)

shape: (288, 8)
┌──────────┬────────────┬──────────────┬──────────┬───────────────┬────────┬───────────────┬───────┐
│ ticker   ┆ queryCount ┆ resultsCount ┆ adjusted ┆ results       ┆ status ┆ request_id    ┆ count │
│ ---      ┆ ---        ┆ ---          ┆ ---      ┆ ---           ┆ ---    ┆ ---           ┆ ---   │
│ str      ┆ i64        ┆ i64          ┆ bool     ┆ struct[10]    ┆ str    ┆ str           ┆ i64   │
╞══════════╪════════════╪══════════════╪══════════╪═══════════════╪════════╪═══════════════╪═══════╡
│ C:EURUSD ┆ 1437       ┆ 288          ┆ true     ┆ {1.09164,1.09 ┆ OK     ┆ b009b3d17a92f ┆ 288   │
│          ┆            ┆              ┆          ┆ 186,["sydney" ┆        ┆ 7ce69010fa38d ┆       │
│          ┆            ┆              ┆          ┆ , "tok...     ┆        ┆ 3da21a        ┆       │
│ C:EURUSD ┆ 1437       ┆ 288          ┆ true     ┆ {1.09148,1.09 ┆ OK     ┆ b009b3d17a92f ┆ 288   │
│          ┆            ┆              ┆          ┆ 177,["sydney" ┆        

In [6]:
res = t['results']
print(res['c'])

ValueError: Cannot __getitem__ on Series of dtype: 'Struct([Field('c': Float64), Field('h': Float64), Field('i_session': List(Utf8)), Field('l': Float64), Field('n': Int64), Field('o': Float64), Field('t': Int64), Field('ts': Utf8), Field('v': Int64), Field('vw': Float64)])' with argument: 'c' of type: '<class 'str'>'.

In [10]:
# close, high, low, # txn, open, timestamp, volume, vwap
res = t['results']
l = pl.DataFrame(map(lambda x: { 'i': x['i_session'], 'c': x['c'], 'h': x['h'], 'l': x['l'], 'n': x['n'], 'o': x['o'], 't': x['t'], 'ts': datetime.utcfromtimestamp(x['t']/1000), 'v': x['v'], 'vw': x['vw'], }, res))
print(l.sort('ts', reverse=True))


print(f"high: {l['h'].max()}")
print(f"low: {l['l'].min()}")

shape: (288, 10)
┌────────────┬─────────┬─────────┬────────┬─────┬───────────────┬──────────────┬─────┬────────┐
│ i          ┆ c       ┆ h       ┆ l      ┆ ... ┆ t             ┆ ts           ┆ v   ┆ vw     │
│ ---        ┆ ---     ┆ ---     ┆ ---    ┆     ┆ ---           ┆ ---          ┆ --- ┆ ---    │
│ list[str]  ┆ f64     ┆ f64     ┆ f64    ┆     ┆ i64           ┆ datetime[μs] ┆ i64 ┆ f64    │
╞════════════╪═════════╪═════════╪════════╪═════╪═══════════════╪══════════════╪═════╪════════╡
│ ["sydney"] ┆ 1.0995  ┆ 1.0999  ┆ 1.0995 ┆ ... ┆ 1681343700000 ┆ 2023-04-12   ┆ 274 ┆ 1.0997 │
│            ┆         ┆         ┆        ┆     ┆               ┆ 23:55:00     ┆     ┆        │
│ ["sydney"] ┆ 1.09986 ┆ 1.1     ┆ 1.0996 ┆ ... ┆ 1681343400000 ┆ 2023-04-12   ┆ 331 ┆ 1.0998 │
│            ┆         ┆         ┆        ┆     ┆               ┆ 23:50:00     ┆     ┆        │
│ ["sydney"] ┆ 1.09992 ┆ 1.1     ┆ 1.0995 ┆ ... ┆ 1681343100000 ┆ 2023-04-12   ┆ 345 ┆ 1.0998 │
│            ┆         

In [20]:
session = l.select("i")
len(session)
session


288

In [ ]:
import plotly.graph_objects as go
import plotly.express as px

fig = go.Figure(
    data=[
        go.Candlestick(
            x=l['ts'],
            open=l['o'],
            high=l['h'],
            low=l['l'],
            close=l['c'])
    ]
)
fig.show()
px.line(
    l.to_pandas(),
    x='ts',
    y=['c', 'h', 'o', 'l']
)